# Stanford CME 241 (Winter 2021) - Assignment 8

### Bank Owner

Since there are three cases in the question, we need to consider them separately for markov devision process modeling.
1. In the case of selling the entire quantity of stock purchased in the previous day, the states can be written as $ (t, x_t, l_t, b_t, s_t) $
<br> where $ t \in \{1,2,3, ...T\}$ (date), $x_t \in \mathbb{R}_{\geq 0}$ (net cash), $l_t \in \mathbb{R}_{\geq 0}$ (liability), $b_t \in \mathbb{R}_{\geq 0}$ (unfulfilled withdrawal requests), $s_t \in \mathbb{R}_{\geq 0}$ (stock value)
    - The key consideration in the problem is that $c_t \geq c_{min}$ where $c_{min} = K cot(\frac{\pi c_{min}}{2C})$
2. In the case of deciding to increase/reduce the liability
<br> If we let $y_t \in \mathbb{R}$ as the amount of change in liability, $y_t$ has the following constraints.
    - $y_t \geq - l_t$
    - $ y_t \geq c_{min} - x_t$
<br> Thus, we can conclude that $ y_t \geq max(-l_t, c_{min} - x_t)$
3. In the case of deciding to purchase a certain quantity of stock
<br> If we let $z_t \in \mathbb{R}_{\geq 0}$ as the number of stock shares to purchase.
    - $ 0 \leq z_t \leq \frac{x_t + y_t - c_{min}}{s_t} $

In this MDP, action is defined as $ (y_t, z_t) $
<br> where $ t < T, y_T = z_T = 0$
<br> The state transitions can be defined as
$$ x_{t+1} = max(x_t + y_t - z_t s_t - K cot(\frac{\pi min(x_t + y_t - z_t s_t, C)}{2C}) + d_{t+1} - f((t_1, b_t),0) $$
$$ b_{t+1} = max(-x_t - y_t + z_t s_t + K cot(\frac{\pi min(x_t + y_t - z_t s_t, C)}{2C}) - d_{t+1} + f((t_1, b_t),0) $$
$$ s_{t+1} = g(t+1, s_t) $$
where $d_t \in \mathbb{R}_{\geq 0}$ denotes the deposits on day $t$, $w_t = f(t, b_{t-1}), 2 \leq t \leq T$ denotes the withdrawal requests on day $t$, $s_t = g(t, s_{t-1})$ denotes the stock value on day $t$
<br> The reawrd on day $t, 1 \leq t \leq T-1 $ is 0, while the reward on day $T$ is $U(x_T - l_T)$

### Call/Put Options

In [8]:
def call(spot_price, strike_price, maturity_time, current_time, risk_free, sigma):
    U = np.exp(sigma * np.sqrt(current_time/maturity_time))
    D = np.exp(-sigma * np.sqrt(current_time/maturity_time))
    P = (np.exp(risk_free * current_time/maturity_time)-D)/(U-D)
    values = np.zeros((T+1, T+1))
    values[0,0] = S
    for x in range(1, T+1):
        values[x, 0] = values[x-1,0] * U
        for y in range(1, x+1):
            values[x,y] = values[x-1, y-1] * D
    option_value = np.zeros_like(values)
    for x in range(T+1):
        option_value[T,x] = max(0, values[T,x]-strike_price)
    for x in range(T-1, -1, -1):
        for y in range(x+1):
            option_value[x,y] = max(0,values[x,y]-strike_price, np.exp(-risk_free * (current_time/maturity_time))*(P * option_value[x+1,y]+(1-P)*option_value[x+1,y+1]))
    return option_value[0,0]

In [9]:
def put(spot_price, strike_price, maturity_time, current_time, risk_free, sigma):
    U = np.exp(sigma * np.sqrt(current_time/maturity_time))
    D = np.exp(-sigma * np.sqrt(current_time/maturity_time))
    P = (np.exp(risk_free * current_time/maturity_time)-D)/(U-D)
    values = np.zeros((T+1, T+1))
    values[0,0] = S
    for x in range(1, T+1):
        values[x, 0] = values[x-1,0] * U
        for y in range(1, x+1):
            values[x,y] = values[x-1, y-1] * D
    option_value = np.zeros_like(values)
    for x in range(T+1):
        option_value[T,x] = max(0, strike_price - values[T,x])
    for x in range(T-1, -1, -1):
        for y in range(x+1):
            option_value[x,y] = max(0,strike_price-values[x,y], np.exp(-risk_free * (current_time/maturity_time))*(P * option_value[x+1,y]+(1-P)*option_value[x+1,y+1]))
    return option_value[0,0]    